# Metachromatosis
Data taken from [Bowen ME, et al. (2011) Loss-of-function mutations in PTPN11 cause metachondromatosis, but not Ollier disease or Maffucci syndrome. PLoS Genet 7:e1002050](https://pubmed.ncbi.nlm.nih.gov/21533187/).

In metachondromatosis (MC), patients develop tumors both on and within bones. These tumors are called exostoses (on the surface of bone) or endosteal tumors (within bone).

Clinical details are not provided in this pub;ication, but from the description we infer that each affected individual had the following:
- [Multiple enchondromatosis - HP:0005701](https://hpo.jax.org/app/browse/term/HP:0005701)
- [Multiple exostoses - HP:0002762](https://hpo.jax.org/app/browse/term/HP:0002762)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import CohortValidator
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.8.31


In [2]:
PMID = "PMID:21533187"
title = "Loss-of-function mutations in PTPN11 cause metachondromatosis, but not Ollier disease or Maffucci syndrome"
citation = Citation(pmid=PMID, title=title)
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=citation)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
# With respect to transcript NM_002834.5 of PTPN11
ptpn11_variants = {
    "family A": "c.409_413del",
    "family B":"c.458_463delinsAAGAACACAGGGGAGAGCA",
    "family C": "c.353_354del",
    "family I": "c.295A>T",
    "family D":"c.1315del",
    "family E": "c.1516C>T",
    "family F": "c.643-2A>C",
    "family G": "c.1093-1G>T",
    "family H": "c.680_683del",
    "family O": "c.1225-1G>A"
}

metachondromatosis = Disease(disease_id='OMIM:156250', disease_label='Metachondromatosis')


default_genotype = 'heterozygous'
transcript='NM_002834.5'
vvalidator = VariantValidator(genome_build="hg38", transcript=transcript)
variant_d = {}
for v in ptpn11_variants.values():
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.409_413del/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.458_463delinsAAGAACACAGGGGAGAGCA/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.353_354del/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.295A>T/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.1315del/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.1516C>T/NM_002834.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002834.5%3Ac.643-2A>C/NM_002834.5?content-type=a

In [4]:
individuals = []

menchondromatosis = HpTerm(hpo_id="HP:0005701", label="Multiple enchondromatosis")
mexostoses = HpTerm(hpo_id="HP:0002762", label="Multiple exostoses")
hpo_terms = [menchondromatosis, mexostoses]

for pat_id, variant in ptpn11_variants.items():
    variant = variant_d.get(variant)
    variant.set_heterozygous()
    #print(variant.to_string())
    vinterpretation = variant.to_ga4gh_variant_interpretation()
    indi = Individual(individual_id=pat_id,
                      hpo_terms=hpo_terms,
                      citation=citation,
                      interpretation_list=[vinterpretation],
                      disease=metachondromatosis
                     )
    individuals.append(indi)
print(f"Got {len(individuals)} individuals")

Got 10 individuals


In [5]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(ontology=hpo_ontology, cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [6]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
family A (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.409_413del (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family B (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.458_463delinsAAGAACACAGGGGAGAGCA (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family C (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.353_354del (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family I (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.295A>T (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family D (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.1315del (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family E (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.1516C>T (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family F (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.643-2A>C (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family G (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.1093-1G>T (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family H (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.680_683del (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)
family O (UNKNOWN; ),Metachondromatosis (OMIM:156250),NM_002834.5:c.1225-1G>A (heterozygous),Multiple enchondromatosis (HP:0005701); Multiple exostoses (HP:0002762)


In [7]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata)

We output 10 GA4GH phenopackets to the directory phenopackets
